In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    e_next_states_logits, eQs = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size)
    a_next_states_logits, aQs = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                    action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=e_next_states_logits, # GQN
                                                                   labels=next_states_labels))
#     eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
#                                                                      labels=next_states_labels)) # maximize loss
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=a_next_states_logits, 
                                                                    labels=next_states_labels)) # minimize loss
    eQs_logits = tf.reshape(eQs, shape=[-1])
    aQs_logits = tf.reshape(aQs, shape=[-1])
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _, aQs2 = Env(actions=next_actions_logits, states=next_states, hidden_size=hidden_size, 
                  action_size=action_size, state_size=state_size, reuse=True)
    aQs2_logits = tf.reshape(aQs2, shape=[-1]) * (1-dones)
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=eQs_logits, # GAN
                                                                    labels=rates)) # 0-1 real
#     eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                     labels=tf.zeros_like(rates))) # min
#     aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=(aQs_logits+aQs2_logits)/2, # GAN
#                                                                      labels=tf.ones_like(rates))) # max
    ###################################################### Q(s,a)= r + Q'(s',a')
    targetQs = rewards + (gamma * aQs2_logits) # DQN/Qlearning
    eloss += tf.reduce_mean(tf.square(eQs_logits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(aQs_logits - rewards)) # min
    aloss2 += tf.reduce_mean(tf.square(aQs_logits - targetQs)) # max
#     eloss += tf.reduce_mean((aQs_logits+aQs2_logits)/2) # min
#     aloss2 += -tf.reduce_mean((aQs_logits+aQs2_logits)/2) # max
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:13.0000 R:13.0000 rate:0.0260 aloss:0.7280 eloss:3.6069 aloss2:1.7069 exploreP:0.9987
Episode:1 meanR:23.5000 R:34.0000 rate:0.0680 aloss:0.7171 eloss:3.5316 aloss2:1.7099 exploreP:0.9954
Episode:2 meanR:21.0000 R:16.0000 rate:0.0320 aloss:0.7248 eloss:3.4215 aloss2:1.7104 exploreP:0.9938
Episode:3 meanR:22.2500 R:26.0000 rate:0.0520 aloss:0.7216 eloss:3.5284 aloss2:1.7059 exploreP:0.9912
Episode:4 meanR:29.4000 R:58.0000 rate:0.1160 aloss:0.7254 eloss:3.3937 aloss2:1.7040 exploreP:0.9856
Episode:5 meanR:26.6667 R:13.0000 rate:0.0260 aloss:0.7192 eloss:3.4125 aloss2:1.7004 exploreP:0.9843
Episode:6 meanR:25.2857 R:17.0000 rate:0.0340 aloss:0.7230 eloss:3.3392 aloss2:1.7211 exploreP:0.9826
Episode:7 meanR:25.1250 R:24.0000 rate:0.0480 aloss:0.7196 eloss:3.3359 aloss2:1.6999 exploreP:0.9803
Episode:8 meanR:24.2222 R:17.0000 rate:0.0340 aloss:0.7300 eloss:3.1936 aloss2:1.7084 exploreP:0.9787
Episode:9 meanR:23.4000 R:16.0000 rate:0.0320 aloss:0.7123 eloss:3.3498 aloss2:1.7

Episode:80 meanR:21.6049 R:23.0000 rate:0.0460 aloss:0.6902 eloss:2.6047 aloss2:1.6790 exploreP:0.8411
Episode:81 meanR:22.0610 R:59.0000 rate:0.1180 aloss:0.6881 eloss:2.5987 aloss2:1.6778 exploreP:0.8362
Episode:82 meanR:22.1566 R:30.0000 rate:0.0600 aloss:0.6886 eloss:2.5891 aloss2:1.6750 exploreP:0.8337
Episode:83 meanR:22.3214 R:36.0000 rate:0.0720 aloss:0.6883 eloss:2.6114 aloss2:1.6787 exploreP:0.8307
Episode:84 meanR:22.3176 R:22.0000 rate:0.0440 aloss:0.6868 eloss:2.6206 aloss2:1.6782 exploreP:0.8289
Episode:85 meanR:22.2093 R:13.0000 rate:0.0260 aloss:0.6862 eloss:2.6175 aloss2:1.6826 exploreP:0.8279
Episode:86 meanR:22.3678 R:36.0000 rate:0.0720 aloss:0.6884 eloss:2.6170 aloss2:1.6729 exploreP:0.8249
Episode:87 meanR:22.3182 R:18.0000 rate:0.0360 aloss:0.6875 eloss:2.6249 aloss2:1.6790 exploreP:0.8235
Episode:88 meanR:22.2809 R:19.0000 rate:0.0380 aloss:0.6880 eloss:2.6349 aloss2:1.6823 exploreP:0.8219
Episode:89 meanR:22.2000 R:15.0000 rate:0.0300 aloss:0.6869 eloss:2.6391 

Episode:159 meanR:28.0700 R:25.0000 rate:0.0500 aloss:0.6803 eloss:2.7164 aloss2:1.6695 exploreP:0.6688
Episode:160 meanR:27.9700 R:16.0000 rate:0.0320 aloss:0.6818 eloss:2.7478 aloss2:1.6735 exploreP:0.6677
Episode:161 meanR:28.6400 R:85.0000 rate:0.1700 aloss:0.6820 eloss:2.7244 aloss2:1.6703 exploreP:0.6622
Episode:162 meanR:28.8600 R:34.0000 rate:0.0680 aloss:0.6817 eloss:2.7079 aloss2:1.6708 exploreP:0.6600
Episode:163 meanR:29.0900 R:36.0000 rate:0.0720 aloss:0.6830 eloss:2.7409 aloss2:1.6702 exploreP:0.6576
Episode:164 meanR:29.4700 R:57.0000 rate:0.1140 aloss:0.6861 eloss:2.7163 aloss2:1.6730 exploreP:0.6539
Episode:165 meanR:29.4600 R:11.0000 rate:0.0220 aloss:0.6807 eloss:2.7331 aloss2:1.6734 exploreP:0.6532
Episode:166 meanR:29.5000 R:25.0000 rate:0.0500 aloss:0.6857 eloss:2.7042 aloss2:1.6698 exploreP:0.6516
Episode:167 meanR:29.6900 R:46.0000 rate:0.0920 aloss:0.6813 eloss:2.7335 aloss2:1.6703 exploreP:0.6487
Episode:168 meanR:30.2500 R:65.0000 rate:0.1300 aloss:0.6802 elo

Episode:238 meanR:65.2000 R:144.0000 rate:0.2880 aloss:0.6792 eloss:2.7008 aloss2:1.6635 exploreP:0.3785
Episode:239 meanR:65.7300 R:67.0000 rate:0.1340 aloss:0.6768 eloss:2.7157 aloss2:1.6661 exploreP:0.3760
Episode:240 meanR:66.9000 R:132.0000 rate:0.2640 aloss:0.6759 eloss:2.7012 aloss2:1.6633 exploreP:0.3712
Episode:241 meanR:67.4100 R:68.0000 rate:0.1360 aloss:0.6773 eloss:2.6981 aloss2:1.6643 exploreP:0.3688
Episode:242 meanR:68.6000 R:158.0000 rate:0.3160 aloss:0.6739 eloss:2.7062 aloss2:1.6650 exploreP:0.3632
Episode:243 meanR:69.4500 R:108.0000 rate:0.2160 aloss:0.6764 eloss:2.7003 aloss2:1.6627 exploreP:0.3594
Episode:244 meanR:71.0000 R:173.0000 rate:0.3460 aloss:0.6769 eloss:2.6993 aloss2:1.6612 exploreP:0.3534
Episode:245 meanR:72.0400 R:117.0000 rate:0.2340 aloss:0.6811 eloss:2.6980 aloss2:1.6577 exploreP:0.3494
Episode:246 meanR:73.8100 R:188.0000 rate:0.3760 aloss:0.6797 eloss:2.7086 aloss2:1.6624 exploreP:0.3431
Episode:247 meanR:75.0200 R:130.0000 rate:0.2600 aloss:0.

Episode:316 meanR:126.3900 R:232.0000 rate:0.4640 aloss:0.6655 eloss:2.6544 aloss2:1.6564 exploreP:0.1364
Episode:317 meanR:131.0100 R:483.0000 rate:0.9660 aloss:0.6625 eloss:2.6479 aloss2:1.6570 exploreP:0.1305
Episode:318 meanR:132.5300 R:174.0000 rate:0.3480 aloss:0.6638 eloss:2.6540 aloss2:1.6538 exploreP:0.1284
Episode:319 meanR:133.1400 R:187.0000 rate:0.3740 aloss:0.6596 eloss:2.6401 aloss2:1.6594 exploreP:0.1262
Episode:320 meanR:134.2400 R:174.0000 rate:0.3480 aloss:0.6607 eloss:2.6438 aloss2:1.6579 exploreP:0.1242
Episode:321 meanR:133.6800 R:118.0000 rate:0.2360 aloss:0.6610 eloss:2.6464 aloss2:1.6584 exploreP:0.1229
Episode:322 meanR:136.3700 R:315.0000 rate:0.6300 aloss:0.6647 eloss:2.6506 aloss2:1.6562 exploreP:0.1194
Episode:323 meanR:136.6500 R:149.0000 rate:0.2980 aloss:0.6599 eloss:2.6635 aloss2:1.6538 exploreP:0.1177
Episode:324 meanR:137.4500 R:207.0000 rate:0.4140 aloss:0.6628 eloss:2.6640 aloss2:1.6565 exploreP:0.1155
Episode:325 meanR:138.0700 R:109.0000 rate:0.2

...

Episode:1172 meanR:467.7700 R:500.0000 rate:1.0000 aloss:0.3582 eloss:0.8633 aloss2:2.0374 exploreP:0.0100
Episode:1173 meanR:472.5500 R:500.0000 rate:1.0000 aloss:0.3522 eloss:0.8614 aloss2:2.0184 exploreP:0.0100
Episode:1174 meanR:472.5500 R:500.0000 rate:1.0000 aloss:0.3525 eloss:0.8625 aloss2:2.0227 exploreP:0.0100
Episode:1175 meanR:472.5500 R:500.0000 rate:1.0000 aloss:0.3498 eloss:0.8660 aloss2:2.0276 exploreP:0.0100
Episode:1176 meanR:472.5500 R:500.0000 rate:1.0000 aloss:0.3459 eloss:0.8606 aloss2:2.0158 exploreP:0.0100
Episode:1177 meanR:472.5500 R:500.0000 rate:1.0000 aloss:0.3428 eloss:0.8605 aloss2:2.0098 exploreP:0.0100
Episode:1178 meanR:472.5500 R:500.0000 rate:1.0000 aloss:0.3425 eloss:0.8637 aloss2:2.0233 exploreP:0.0100
Episode:1179 meanR:477.3500 R:500.0000 rate:1.0000 aloss:0.3412 eloss:0.8632 aloss2:2.0248 exploreP:0.0100
Episode:1180 meanR:477.3500 R:500.0000 rate:1.0000 aloss:0.3385 eloss:0.8611 aloss2:2.0172 exploreP:0.0100
Episode:1181 meanR:477.3500 R:500.000

Episode:1249 meanR:403.3200 R:90.0000 rate:0.1800 aloss:0.3093 eloss:0.8693 aloss2:2.2696 exploreP:0.0100
Episode:1250 meanR:402.8900 R:457.0000 rate:0.9140 aloss:0.3028 eloss:0.8814 aloss2:2.2733 exploreP:0.0100
Episode:1251 meanR:402.8900 R:500.0000 rate:1.0000 aloss:0.3033 eloss:0.9930 aloss2:2.3261 exploreP:0.0100
Episode:1252 meanR:398.1500 R:26.0000 rate:0.0520 aloss:0.3067 eloss:0.8887 aloss2:2.3028 exploreP:0.0100
Episode:1253 meanR:398.1500 R:500.0000 rate:1.0000 aloss:0.3015 eloss:0.8944 aloss2:2.2932 exploreP:0.0100
Episode:1254 meanR:393.3900 R:24.0000 rate:0.0480 aloss:0.2995 eloss:0.8693 aloss2:2.2851 exploreP:0.0100
Episode:1255 meanR:388.9000 R:51.0000 rate:0.1020 aloss:0.3004 eloss:0.8760 aloss2:2.2928 exploreP:0.0100
Episode:1256 meanR:388.9000 R:500.0000 rate:1.0000 aloss:0.3020 eloss:0.8874 aloss2:2.3054 exploreP:0.0100
Episode:1257 meanR:384.3100 R:41.0000 rate:0.0820 aloss:0.2976 eloss:0.8772 aloss2:2.3090 exploreP:0.0100
Episode:1258 meanR:379.7100 R:40.0000 rate

Episode:1326 meanR:267.5300 R:10.0000 rate:0.0200 aloss:0.2921 eloss:1.4532 aloss2:3.5884 exploreP:0.0100
Episode:1327 meanR:268.2900 R:313.0000 rate:0.6260 aloss:0.2908 eloss:1.2073 aloss2:3.1197 exploreP:0.0100
Episode:1328 meanR:263.3900 R:10.0000 rate:0.0200 aloss:0.2720 eloss:1.1840 aloss2:2.9781 exploreP:0.0100
Episode:1329 meanR:260.9600 R:10.0000 rate:0.0200 aloss:0.2888 eloss:1.2101 aloss2:3.4518 exploreP:0.0100
Episode:1330 meanR:262.1800 R:270.0000 rate:0.5400 aloss:0.2908 eloss:1.1914 aloss2:3.0490 exploreP:0.0100
Episode:1331 meanR:263.4900 R:222.0000 rate:0.4440 aloss:0.2932 eloss:1.2340 aloss2:3.1676 exploreP:0.0100
Episode:1332 meanR:261.0800 R:259.0000 rate:0.5180 aloss:0.2871 eloss:1.2563 aloss2:3.2039 exploreP:0.0100
Episode:1333 meanR:263.8800 R:429.0000 rate:0.8580 aloss:0.2873 eloss:1.5175 aloss2:3.6365 exploreP:0.0100
Episode:1334 meanR:261.0500 R:217.0000 rate:0.4340 aloss:0.2876 eloss:1.2160 aloss2:3.2716 exploreP:0.0100
Episode:1335 meanR:258.3300 R:228.0000 r

Episode:1403 meanR:142.3600 R:11.0000 rate:0.0220 aloss:0.2682 eloss:1.6325 aloss2:3.7721 exploreP:0.0100
Episode:1404 meanR:139.7400 R:36.0000 rate:0.0720 aloss:0.3260 eloss:3.0844 aloss2:4.9796 exploreP:0.0100
Episode:1405 meanR:138.1500 R:10.0000 rate:0.0200 aloss:0.2956 eloss:2.9350 aloss2:4.9469 exploreP:0.0100
Episode:1406 meanR:136.8100 R:10.0000 rate:0.0200 aloss:0.2702 eloss:2.0661 aloss2:3.8123 exploreP:0.0100
Episode:1407 meanR:139.6000 R:289.0000 rate:0.5780 aloss:0.2787 eloss:3.2293 aloss2:4.6860 exploreP:0.0100
Episode:1408 meanR:144.3700 R:487.0000 rate:0.9740 aloss:0.2709 eloss:2.8148 aloss2:4.4060 exploreP:0.0100
Episode:1409 meanR:146.8100 R:255.0000 rate:0.5100 aloss:0.2675 eloss:3.9310 aloss2:4.5843 exploreP:0.0100
Episode:1410 meanR:148.0800 R:415.0000 rate:0.8300 aloss:0.2727 eloss:5.0259 aloss2:5.3138 exploreP:0.0100
Episode:1411 meanR:144.9900 R:11.0000 rate:0.0220 aloss:0.2715 eloss:1.4319 aloss2:3.8493 exploreP:0.0100
Episode:1412 meanR:142.6800 R:11.0000 rate

Episode:1480 meanR:258.3200 R:500.0000 rate:1.0000 aloss:0.4589 eloss:1.4976 aloss2:2.5604 exploreP:0.0100
Episode:1481 meanR:263.2200 R:500.0000 rate:1.0000 aloss:0.4622 eloss:1.4636 aloss2:2.6019 exploreP:0.0100
Episode:1482 meanR:265.1500 R:500.0000 rate:1.0000 aloss:0.4557 eloss:1.4458 aloss2:2.5364 exploreP:0.0100


# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
